In [ ]:
# Required to use in Google Colab
!pip install future
!git clone https://github.com/BubaVV/Pyevolve.git
!cd Pyevolve && python3 setup.py install && mv pyevolve/ ../

In [ ]:
# Imports for this practical work
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Consts
from pyevolve import Crossovers
from pyevolve import Mutators
from pyevolve import Selectors
from pyevolve import DBAdapters
from geopy.distance import geodesic
import matplotlib.pyplot as plt 
import random

In [ ]:
# Cities coordinates
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 
       17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 
       96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

In [ ]:
cities = list(zip(LAT, LON))

In [ ]:
def matrix_dist(list_cities):
  matrix = {}
  for i,c1 in enumerate(list_cities):
    for j,c2 in enumerate(list_cities):
      matrix[i, j] = geodesic(c1, c2).km
  return matrix

matrix_cities = matrix_dist(cities)

In [ ]:
def total_dist_tour(tour):
  total = 0
  t = tour.getInternalList()
  for i in range(len(cities)):
    j = (i + 1) % len(cities)
    total += matrix_cities[t[i], t[j]]
  return total

In [ ]:
def fitness(chromosome):
  return total_dist_tour(chromosome)

In [ ]:
def G1DListTSPInitializator(genome, **args):
  lst = [i for i in range(genome.getListSize())]
  random.shuffle(lst)
  genome.setInternalList(lst)

In [ ]:
# Chromosome representation

# genome = List of elements
genome = G1DList.G1DList(len(cities))

# elements = values in the range rangemin and rangemax
genome.setParams(rangemin=0, rangemax=len(cities) - 1)

# how to compute the fitness
genome.evaluator.set(fitness)
genome.initializator.set(G1DListTSPInitializator)
genome.mutator.set(Mutators.G1DListMutatorSwap)
genome.crossover.set(Crossovers.G1DListCrossoverEdge)

# GA initialisation
ga = GSimpleGA.GSimpleGA(genome)
ga.setPopulationSize(100)
ga.setMutationRate(0.02)
ga.setCrossoverRate(1.0)
ga.setMinimax(Consts.minimaxType["minimize"])
ga.setElitism(True)

# Number of generations
ga.setGenerations(250)

# Record evolution results in a database
sqlite_adapter = DBAdapters.DBSQLite(identify="mlg_ga_tsp", resetDB=True)
ga.setDBAdapter(sqlite_adapter)

# run the GA and print out stats every freq_stats generations
ga.evolve(freq_stats=25)

best = ga.bestIndividual()

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw)             [7988.88(8196.69)/5475.46(5291.00)/6657.40(6657.40)]
Gen. 25 (10.00%): Max/Min/Avg Fitness(Raw)             [5339.24(6413.43)/3977.48(3407.86)/4449.37(4449.37)]
Gen. 50 (20.00%): Max/Min/Avg Fitness(Raw)             [5174.55(7224.94)/4026.30(3346.76)/4312.12(4312.12)]
Gen. 75 (30.00%): Max/Min/Avg Fitness(Raw)             [5076.40(7483.63)/4000.55(3346.76)/4230.33(4230.33)]
Gen. 100 (40.00%): Max/Min/Avg Fitness(Raw)             [5376.48(6586.98)/3998.18(3346.76)/4480.40(4480.40)]
Gen. 125 (50.00%): Max/Min/Avg Fitness(Raw)             [5345.03(6648.40)/4004.58(3346.76)/4454.19(4454.19)]
Gen. 150 (60.00%): Max/Min/Avg Fitness(Raw)             [5379.79(6706.33)/4024.84(3346.76)/4483.16(4483.16)]
Gen. 175 (70.00%): Max/Min/Avg Fitness(Raw)             [5297.53(7613.04)/4119.83(3346.76)/4414.61(4414.61)]
Gen. 200 (80.00%): Max/Min/Avg Fitness(Raw)             [5296.09(6725.89)/4006.26(3346.76)/4413.41(4413.41)]
Gen. 225 (90.00%): Max/Mi

In [ ]:
print(best)
print(best.genomeList)
print(fitness(best))

- GenomeBase
	Score:			 3346.761974
	Fitness:		 3965.758876

	Params:		 {'rangemin': 0, 'rangemax': 13}

	Slot [Evaluator] (Count: 1)
		Name: fitness - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: G1DListTSPInitializator - Weight: 0.50
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  The mutator of G1DList, Swap Mutator

    .. note:: this mutator is :term:`Data Type Independent`

    
	Slot [Crossover] (Count: 1)
		Name: G1DListCrossoverEdge - Weight: 0.50
		Doc:  The Edge Recombination crossover for G1DList (widely used for TSP problem)

    See more information in the `Edge Recombination Operator
    <http://en.wikipedia.org/wiki/Edge_recombination_operator>`_
    Wikipedia entry.
    

- G1DList
	List size:	 14
	List:		 [0, 1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9]


[0, 1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9]
3346.7619740386313


In [ ]:
# https://github.com/perone/Pyevolve/blob/master/pyevolve_graph.py
# -0 display all graphs
%run ./Pyevolve/pyevolve_graph.py -i mlg_ga_tsp -0

Pyevolve 0.6 - Graph Plot Tool
By Christian S. Perone

Loading modules....
Loading database and creating graph...
250 generations found !
Graph saved to graphs_mlg_ga_tsp/mlg_ga_tsp_errorbars_raw.png file !
Graph saved to graphs_mlg_ga_tsp/mlg_ga_tsp_errorbars_fitness.png file !
Graph saved to graphs_mlg_ga_tsp/mlg_ga_tsp_maxmin_raw.png file !
Graph saved to graphs_mlg_ga_tsp/mlg_ga_tsp_maxmin_fitness.png file !
Graph saved to graphs_mlg_ga_tsp/mlg_ga_tsp_diff_raw.png file !

	Done ! The graphs was saved in the directory 'graphs_mlg_ga_tsp'
